In [19]:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import operator

In [ ]:
def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

In [ ]:
print('Load data...')
train = pd.read_csv("./input/train.csv")
target = train['target'].values
train = train.drop(['ID','target'],axis=1)

test = pd.read_csv("./input/test.csv")
id_test = test['ID'].values
test = test.drop(['ID'],axis=1)

In [ ]:
print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.median()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.median()  #TODO

In [ ]:
xgtrain = xgb.DMatrix(train.values, target)
xgtest = xgb.DMatrix(test.values)

In [ ]:
# XGBoost params:
xgboost_params = { 
   "objective": "binary:logistic",
   "booster": "gbtree",
   "eval_metric": "auc",
   "eta": 0.01, # 0.06, #0.01,
   #"min_child_weight": 240,
   "subsample": 0.75,
   "colsample_bytree": 0.68,
   "max_depth": 7
}


In [14]:
print('Fit the model...')
boost_round = 2000 #CHANGE THIS BEFORE START
clf = xgb.train(xgboost_params,xgtrain,num_boost_round=boost_round,verbose_eval=True,maximize=False)

Fit the model...


In [15]:
#Run xgboost parameter importance to see if that is matching with current top
features = list(train)
ceate_feature_map(features)

In [25]:
importance = clf.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df_PI = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_PI['fscore'] = df_PI['fscore'] / df_PI['fscore'].sum()
df_PI= df_PI.sort(['fscore'],ascending=False)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [29]:
rm_List = ['v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73',
                    'v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116',
                    'v117','v118','v119','v123','v124','v128']

In [41]:
df_PI_rm = df_PI[df_PI.feature.isin(rm_List) ]

In [42]:
df_PI_rm

,feature,fscore
114,v79,0.010469
83,v119,0.007231
82,v82,0.007214
76,v123,0.006938
75,v118,0.006938
72,v53,0.006692
71,v36,0.006651
65,v37,0.006486
61,v51,0.006328
60,v124,0.006293
